## Fetches information from Spoontacular API

In [29]:
import requests
from pathlib import Path
import os

In [50]:
mykey = open('spoonac/apikey.txt').read().strip()



In [60]:
#nice command for viewing files: jq . entrees0.json | less
#nice command for keeping an eye on the program while running watch ls -l entrees
cuisines = 'American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,Greek,Indian,Irish,Italian,Japanese,Jewish,Korean,Latin American,Mediterranean,Mexican,Middle Eastern,Nordic,Spanish,Thai,Vietnamese'.split(',')
number = 100

def get_recipes(cuisine):
    rt = f'spoonac/data/{cuisine}'
    pages_got = Path(f'{rt}/got.csv')
    if not pages_got.exists():
        pages_got.parent.mkdir(parents=True, exist_ok=True) 
        with pages_got.open('w') as fh:
            fh.write('0\n')
    
    offset = pages_got.open().read().strip()
    offset = int(offset)
    if offset >= 900:
        print(f'Got all the {cuisine}')
        return 'finished'
        
    
    params = {'number': str(number), 'type': 'main course', 'apiKey': mykey, 'offset': str(offset), 'cuisine': cuisine}
    response = requests.get('https://api.spoonacular.com/recipes/complexSearch', params=params)
    
    numused = float(response.headers['X-API-Quota-Used'])
    if numused >= 4499:
        raise Exception(f'Used {numused} of 4500 points')
    
    with open(f'last_results.json', 'w') as fh:
        fh.write(response.text)
    res = response.json()
    
    ids = [str(recipe['id']) for recipe in res['results']]
    idformatted = ','.join(ids)
    
    params = {'apiKey': mykey, 'ids': idformatted}
    result = requests.get(f'https://api.spoonacular.com/recipes/informationBulk/', params=params)
    if result.status_code >= 300:
        raise Exception(result.text)

    txt = result.text
    with open(f"{rt}/maincourse-{offset}.json", 'w') as fh:
        fh.write(txt)
    offset += len(result.json())

    with open(f"{pages_got}.new", "w") as fh:
        fh.write(f"{offset}\n")
    os.rename(f"{pages_got}.new", pages_got)

    if res['offset']+res['number'] >= res['totalResults']:
        return 'finished'
    return 'running'
    
    
donecuisines = open('spoonac/data/donecuisines.csv').read().strip().split(',')



while True:
    for cuisine in cuisines:
        state = 'running'
        while state == 'running' and cuisine not in donecuisines:
            print(f'Running {cuisine}')
            state = get_recipes(cuisine)
        if state == 'finished':
            donecuisines.append(cuisine)
            out = ','.join(donecuisines)
            print(f'Finished {out}')

            with open('spoonac/data/donecuisines.csv', 'a') as fh:
                fh.write(out)
        
    if len(donecuisines) == len(cuisines):
        break
print('Finished all cuisines')
    
            
            
        
    
    
    

Running Middle Eastern
Running Middle Eastern
Running Middle Eastern
Running Middle Eastern
Finished ,American,American,Southern,American,Southern,French,American,Southern,French,African,American,Southern,French,African,British,American,Southern,French,African,British,Cajun,American,Southern,French,African,British,Cajun,Caribbean,American,Southern,French,African,British,Cajun,Caribbean,Chinese,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,Greek,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,Greek,Indian,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,Greek,Indian,Irish,

Running Vietnamese
Running Vietnamese
Finished ,American,American,Southern,American,Southern,French,American,Southern,French,African,American,Southern,French,African,British,American,Southern,French,African,British,Cajun,American,Southern,French,African,British,Cajun,Caribbean,American,Southern,French,African,British,Cajun,Caribbean,Chinese,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,Greek,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,Greek,Indian,American,Southern,French,African,British,Cajun,Caribbean,Chinese,Eastern European,European,German,Greek,Indian,Irish,American,Southern,French,African,British,Cajun,Caribbe

KeyboardInterrupt: 

In [53]:
params = {'apiKey': mykey}
print(params)
result = requests.get(f'https://api.spoonacular.com/recipes/770426/information', params=params)


{'apiKey': 'f18bdef65484424a937f3b550fec8ce2'}


In [58]:
# for h, v in sorted(result.headers.items()):
#     print(f"{h}: {v}")
    
# print(result.text)

float(result.headers['X-API-Quota-Used'])


1.0

In [ ]:
#X-API-Quota-Used: The number of points used in total today.